# 8.3 SQL Case Study - Country Club


Q1 : Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

SELECT name
FROM Facilities
WHERE membercost > 0

Q2: How many facilities do not charge a fee to members?

SELECT COUNT(*)
FROM Facilities
WHERE membercost = 0

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

SELECT facid,name,membercost,monthlymaintenance
FROM Facilities
WHERE monthlymaintenance * 0.2 > membercost

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

SELECT *
FROM Facilities
WHERE facid =1
UNION
SELECT *
FROM Facilities
WHERE facid =5
ORDER BY facid

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

SELECT name,monthlymaintenance, 
CASE WHEN monthlymaintenance > 100 THEN 'Expensive'
ELSE 'Cheap' END AS Relative_Price
FROM Facilities

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

SELECT surname,firstname FROM Members WHERE joindate = (SELECT Max(joindate) FROM Members)

SELECT CONCAT( m.firstname,' ',m.surname) as name, f.name as Name_of_court
    FROM (
    SELECT memid, facid
    FROM Bookings
    WHERE facid =0
    OR facid =1
    GROUP BY memid
    ) AS l
    LEFT JOIN Members AS m ON m.memid = l.memid
    LEFT JOIN Facilities as f on f.facid = l.facid
    ORDER BY name

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. 

SELECT f.name, b.memid, b.starttime,
case WHEN b.memid =0 THEN m.firstname
ELSE CONCAT(m.firstname,' ',m.surname) END AS Name,
CASE WHEN b.memid =0
THEN f.guestcost
ELSE f.membercost
END AS Cost
FROM Bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
LEFT JOIN Members AS m ON m.memid = b.memid
WHERE EXTRACT( YEAR
FROM starttime ) =2012
AND EXTRACT(
MONTH FROM starttime ) =9
AND EXTRACT(
DAY FROM starttime ) =14 and case WHEN b.memid =0 THEN f.guestcost > 30 ELSE f.membercost>30 END

Q9: This time, produce the same result as in Q8, but using a subquery.

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [11]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT  mc.facid,(f.membercost * mc.total + f.guestcost * gc.total) as Total_Revenue
FROM (

SELECT facid, COUNT( memid ) AS total
FROM Bookings
WHERE memid <>0
GROUP BY facid
) AS mc
LEFT JOIN Facilities AS f ON f.facid = mc.facid
LEFT JOIN (

SELECT facid, COUNT( memid ) AS total
FROM Bookings
WHERE memid =0
GROUP BY facid
) AS gc ON gc.facid = mc.facid
WHERE f.membercost * mc.total + f.guestcost * gc.total < 1000
ORDER BY Total_Revenue
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

(3, 90)
(7, 115)
(8, 265)
(2, 604.5)


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [26]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT a.Member, b.Member AS Recommendation
FROM (

SELECT memid,
CASE WHEN memid =0
THEN "Guest"
ELSE firstname||' '||surname
END AS Member, recommendedby
FROM Members
) AS a
LEFT JOIN (

SELECT memid,
CASE WHEN memid IS NULL
THEN "No Recommendation"
ELSE firstname||' '||surname
END AS Member, recommendedby
FROM Members
) AS b ON b.memid = a.recommendedby
ORDER BY a.Member
"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

('Anna Mackenzie', 'Darren Smith')
('Anne Baker', 'Ponder Stibbons')
('Burton Tracy', None)
('Charles Owen', 'Darren Smith')
('Darren Smith', None)
('Darren Smith', None)
('David Farrell', None)
('David Jones', 'Janice Joplette')
('David Pinker', 'Jemima Farrell')
('Douglas Jones', 'David Jones')
('Erica Crumpet', 'Tracy Smith')
('Florence Bader', 'Ponder Stibbons')
('Gerald Butters', 'Darren Smith')
('Guest', None)
('Henrietta Rumney', 'Matthew Genting')
('Henry Worthington-Smyth', 'Tracy Smith')
('Hyacinth Tupperware', None)
('Jack Smith', 'Darren Smith')
('Janice Joplette', 'Darren Smith')
('Jemima Farrell', None)
('Joan Coplin', 'Timothy Baker')
('John Hunt', 'Millicent Purview')
('Matthew Genting', 'Gerald Butters')
('Millicent Purview', 'Tracy Smith')
('Nancy Dare', 'Janice Joplette')
('Ponder Stibbons', 'Burton Tracy')
('Ramnaresh Sarwin', 'Florence Bader')
('Tim Boothe', 'Tim Rownam')
('Tim Rownam', None)
('Timothy Baker', 'Jemima Farrell')
('Tracy Smith', None)


Q12: Find the facilities with their usage by member, but not guests

In [28]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT facid, COUNT( memid ) AS Member_usage
FROM Bookings
WHERE memid <>0
GROUP BY facid
"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

(0, 308)
(1, 276)
(2, 344)
(3, 385)
(4, 421)
(5, 27)
(6, 195)
(7, 421)
(8, 783)


Q13: Find the facilities usage by month, but not guests

In [31]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT facid, strftime('%m', starttime) AS
MONTH , COUNT( memid ) AS Member_usage
FROM Bookings
WHERE memid <>0
GROUP BY facid, strftime('%m', starttime)
"""
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

(0, '07', 65)
(0, '08', 111)
(0, '09', 132)
(1, '07', 41)
(1, '08', 109)
(1, '09', 126)
(2, '07', 51)
(2, '08', 132)
(2, '09', 161)
(3, '07', 48)
(3, '08', 143)
(3, '09', 194)
(4, '07', 77)
(4, '08', 153)
(4, '09', 191)
(5, '07', 4)
(5, '08', 9)
(5, '09', 14)
(6, '07', 23)
(6, '08', 85)
(6, '09', 87)
(7, '07', 68)
(7, '08', 154)
(7, '09', 199)
(8, '07', 103)
(8, '08', 272)
(8, '09', 408)
